In [ ]:
'''やること
前回用いた「ニュース」のデータセットを用いて、「２クラス分類」をsklearnのSVMで実装する。
'''

'''目次
1.必要なモジュールとデータセットの準備
2.データセットの確認と絞り込み
3.前処理
4.モデル作成と精度確認
5.新しいデータを入れて遊んでみる（大谷選手、130億年以上前に誕生したブラックホールのニュースを分類してみる）
'''

In [1]:
'''1.必要なモジュールとデータセット、2.データセットの確認と絞り込み

2-6までの解説については、「https://dev.classmethod.jp/machine-learning/tf-idf_zakkurimatome_python/」を参照してください。
ざっくり説明しますと、sklearnに用意されている「ニュース」のデータセットを利用します。
このデータセットから「野球」と「宇宙」に関するデータセットのみに絞り込んでいます。
'''

# 1.必要なモジュールのインポート
from sklearn.datasets import fetch_20newsgroups # モジュールのインポート
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42) # twenty_trainという変数にデータセット（トレーニング用）を格納
 

# 2.データの絞り込み(野球と宇宙の文書だけ)
categories = ['rec.sport.baseball', 'sci.space'] # 今回分析対象とするカテゴリーを絞り込む
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42) # 上記で絞り込んだカテゴリーのデータのみを変数に入れる

print(twenty_train.target_names) # カテゴリの確認(ちゃんと絞り込めているか。)
print (len(twenty_train.data)) # 絞り込んだ結果、ニュースデータが何本になったか（1,190本のニュースデータ）
print("\n".join(twenty_train.data[0].split("\n"))) #データの内容を確認(1個の文書の中身を確認)
print(twenty_train.target_names[twenty_train.target[0]]) #データのカテゴリーを確認




['rec.sport.baseball', 'sci.space']
1190
From: sweda@css.itd.umich.edu (Sean Sweda)
Subject: Royals final run total...
Organization: University of Michigan - ITD Consulting and Support
Lines: 22
NNTP-Posting-Host: stimpy.css.itd.umich.edu
X-Newsreader: TIN [version 1.1 PL8]


I've been saying this for quite some time, but being absent from the
net for a while I figured I'd stick my neck out a bit...

The Royals will set the record for fewest runs scored by an AL
team since the inception of the DH rule.  (p.s. any ideas what this is?)

They will fall easily short of 600 runs, that's for damn sure.  I can't
believe these media fools picking them to win the division (like our
Tom Gage of the Detroit News claiming Herk Robinson is some kind of
genius for the trades/aquisitions he's made)

c-ya

Sean


--
Sean Sweda                                      sweda@css.itd.umich.edu
CSS/ITD Consultant			   President, Bob Sura Fan Club
GM/Manager Motor City Marauders
Internet Baseball League				   

In [2]:
'''3.前処理
tf-idfを計算した後に「lsa」で次元削減をしています。
次元削減をする必要性については、「次元の呪いの回避」、「処理時間の高速化」等様々な理由がございます。
「lsa」そのものについてはまだご説明できていないのですが、後日説明予定です。
'''

# 3-1.tf-idf計算(stop words処理を入れたかったので、手法を少し変えました)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

tfidfv = TfidfVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS) # stop word処理を追加（）

tfv_vector_fit = tfidfv.fit(twenty_train.data)
tfv_vector = tfv_vector_fit.transform(twenty_train.data)
print(tfv_vector.shape) # (1190, 21323)


# 3-2.次元削減(「lsa」を使って次元削減を行います)
from sklearn.decomposition import TruncatedSVD

# 3-2-1.パラメータの調整
list_n_comp = [5,10,50,100,500,1000,5000]
for i in list_n_comp:
    lsa = TruncatedSVD(n_components=i,n_iter=5, random_state = 0)
    lsa.fit(tfv_vector) 
    tfv_vector_lsa = lsa.transform(tfv_vector)
    print('次元削減後の特徴量が{0}の時の説明できる分散の割合合計は{1}です'.format(i,round((sum(lsa.explained_variance_ratio_)),2)))

# 3-2-2.次元削減した状態のデータを作成
# 上記で確認した「n_components」に指定した上で、次元削減（特徴抽出）を行う
lsa = TruncatedSVD(n_components=1000,n_iter=5, random_state = 0)
lsa.fit(tfv_vector) 
tfv_vector_lsa = lsa.transform(tfv_vector)

(1190, 21323)
次元削減後の特徴量が5の時の説明できる分散の割合合計は0.03570709119260532です
次元削減後の特徴量が10の時の説明できる分散の割合合計は0.06447746559282569です
次元削減後の特徴量が50の時の説明できる分散の割合合計は0.20619295833298742です
次元削減後の特徴量が100の時の説明できる分散の割合合計は0.32238360271415295です
次元削減後の特徴量が500の時の説明できる分散の割合合計は0.7673790999616275です
次元削減後の特徴量が1000の時の説明できる分散の割合合計は0.9775822044125919です
次元削減後の特徴量が5000の時の説明できる分散の割合合計は0.9999999999999998です


In [32]:
list_n_comp = [5,10,50,100,500,1000,5000]
for i in list_n_comp:
    lsa = TruncatedSVD(n_components=i,n_iter=5, random_state = 0)
    lsa.fit(tfv_vector) 
    tfv_vector_lsa = lsa.transform(tfv_vector)
    print('次元削減後の特徴量が{0}の時の説明できる分散の割合合計は{1}です'.format(i,round((sum(lsa.explained_variance_ratio_)),2)))

次元削減後の特徴量が5の時の説明できる分散の割合合計は0.04です
次元削減後の特徴量が10の時の説明できる分散の割合合計は0.06です
次元削減後の特徴量が50の時の説明できる分散の割合合計は0.21です
次元削減後の特徴量が100の時の説明できる分散の割合合計は0.32です
次元削減後の特徴量が500の時の説明できる分散の割合合計は0.77です
次元削減後の特徴量が1000の時の説明できる分散の割合合計は0.98です
次元削減後の特徴量が5000の時の説明できる分散の割合合計は1.0です


In [33]:
'''4.モデル作成と精度確認
今回実装する「C-SVM」では「識別超平面の係数（汎化性能につながる）」と「誤分類をどれだけ許容するか」の２つの要素が調整すべきパラメータです。
しかし、実際に人間が指定して調整する必要があるのは「誤分類をどれだけ許容するか」だけです。
（コード中では「C」と表記しています）

なので、この「誤分類をどれだけ許容するか」を「GridSearch」を使って調整します。
調整といってもどのくらいの値にすればいいのかもよくわからないので、最初は大きなレンジでざっくり精度を確認して見ていき、徐々にレンジを絞っていきます。
'''


# 4-1.ハイパーパラメータの調整(１回目)

from sklearn.grid_search import GridSearchCV # 4-1.モジュールのインポート
from sklearn import svm # 4-2.モジュールのインポート

'''パラメータ調整（１回目）
Cが大きいほど「誤分類を許さない」
Cが大きくなると、「係数パラメータの値が大きくなる」≒「汎化性能が低下する可能性が高まる」
パラメータがどのくらいだといいのかの勘所がないので、最初は大きめのレンジで確認します。
'''
tuned_parameters = [{'C':  [1e-4,1e-3, 1e-2, 1e-1, 1, 10, 100, 1000,10000,100000]}]# 4-3.確認するパラメータを指定

# 上記で用意したパラメーターごとに交差検証を実施。次のパラメータの調整のレンジを確認する。
clf = GridSearchCV(svm.SVC(), tuned_parameters, scoring="accuracy",cv=5, n_jobs=-1)
clf.fit(tfv_vector_lsa, twenty_train.target) # 5-3.学習
for params, mean_score, all_scores in clf.grid_scores_:
        print ("{0},精度:{1} ,標準誤差=(+/- {2}) ".format(params, round((mean_score),3), round((all_scores.std() / 2),3))) # 各パラメータごとの精度を確認

{'C': 0.0001},精度:0.5016806722689076 ,標準誤差=(+/- 0.0004201940292178824) 
{'C': 0.001},精度:0.5016806722689076 ,標準誤差=(+/- 0.0004201940292178824) 
{'C': 0.01},精度:0.5016806722689076 ,標準誤差=(+/- 0.0004201940292178824) 
{'C': 0.1},精度:0.5016806722689076 ,標準誤差=(+/- 0.0004201940292178824) 
{'C': 1},精度:0.5016806722689076 ,標準誤差=(+/- 0.0004201940292178824) 
{'C': 10},精度:0.5016806722689076 ,標準誤差=(+/- 0.0004201940292178824) 
{'C': 100},精度:0.9840336134453781 ,標準誤差=(+/- 0.00407247225404466) 
{'C': 1000},精度:0.992436974789916 ,標準誤差=(+/- 0.0030814616539893156) 
{'C': 10000},精度:0.992436974789916 ,標準誤差=(+/- 0.0030814616539893156) 
{'C': 100000},精度:0.992436974789916 ,標準誤差=(+/- 0.0030814616539893156) 


In [36]:
for params, mean_score, all_scores in clf.grid_scores_:
        print ("{0},精度:{1} ,標準誤差=(+/- {2}) ".format(params, round((mean_score),3), round((all_scores.std() / 2),3))) # 各パラメータごとの精度を確認

{'C': 0.0001},精度:0.502 ,標準誤差=(+/- 0.0) 
{'C': 0.001},精度:0.502 ,標準誤差=(+/- 0.0) 
{'C': 0.01},精度:0.502 ,標準誤差=(+/- 0.0) 
{'C': 0.1},精度:0.502 ,標準誤差=(+/- 0.0) 
{'C': 1},精度:0.502 ,標準誤差=(+/- 0.0) 
{'C': 10},精度:0.502 ,標準誤差=(+/- 0.0) 
{'C': 100},精度:0.984 ,標準誤差=(+/- 0.004) 
{'C': 1000},精度:0.992 ,標準誤差=(+/- 0.003) 
{'C': 10000},精度:0.992 ,標準誤差=(+/- 0.003) 
{'C': 100000},精度:0.992 ,標準誤差=(+/- 0.003) 


In [37]:
'''
上記の結果、Cが100より大きいといい感じですね...
また、1000以上にしても変化はなさそうなので、この辺りでもう一回パラメータを調整してみます。
'''

# 4-2.ハイパーパラメータの調整(2回目)
tuned_parameters = [{'C':  [ 30, 50, 100, 300, 500, 700, 1000]}]

clf = GridSearchCV(svm.SVC(), tuned_parameters, scoring="accuracy",cv=5, n_jobs=-1)
clf.fit(tfv_vector_lsa, twenty_train.target) # 5-3.学習
for params, mean_score, all_scores in clf.grid_scores_:
        print ("{0},精度:{1} ,標準誤差=(+/- {2}) ".format(params, round((mean_score),3), round((all_scores.std() / 2),3))) # 3-2-3.各パラメータごとの精度を確認



{'C': 30},精度:0.856 ,標準誤差=(+/- 0.017) 
{'C': 50},精度:0.972 ,標準誤差=(+/- 0.006) 
{'C': 100},精度:0.984 ,標準誤差=(+/- 0.004) 
{'C': 300},精度:0.991 ,標準誤差=(+/- 0.002) 
{'C': 500},精度:0.992 ,標準誤差=(+/- 0.003) 
{'C': 700},精度:0.992 ,標準誤差=(+/- 0.003) 
{'C': 1000},精度:0.992 ,標準誤差=(+/- 0.003) 


In [64]:
# まあ、ここまできたらもういいんじゃないですかね。精度も99%まできていて、これ以上追求するのもなんだし
# 今回はC=300くらいでモデルを作成します。

# 4-3.SVMモデル作成
clf = svm.SVC(C=300)
clf.fit(tfv_vector_lsa, twenty_train.target) # 5-3.学習


SVC(C=300, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
'''5.新しいデータを入れて遊んでみる
（大谷選手、130億年以上前に誕生したブラックホールのニュースを分類してみる）

'''

# 5--1.基礎確認(とても短い文書を入れてみて、簡単にモデルの確認をする)

test = ['i love baseball','I go to the baseball field' ,'i want to be an astronaut', 'Work at NASA']

# 前処理
test_tfv = tfv_vector_fit.transform(test) # tf-idf計算
test_lsa = lsa.transform(test_tfv) # lsaで次元削減

# 予測
test_predict = clf.predict(test_lsa)
print(test_predict)



[0 0 1 1]


In [27]:
'''
yahooニュースから大谷選手がエンゼルス入団を決定した記事を２つチョイスして、google翻訳で英語にしたものを利用しています。
大谷選手のニュースは「０」というクラスに、ブラックホールについてのニュースは「１」というクラスに分類されています。
先ほど確認した結果と比較すると問題なく分類されていそうです。
'''

# 5-2.yahooニュースとgoogle翻訳を使った確認
# 5-2-1.データの取り込み
test = [
'Nippon Ham\'s Shohei Otani pitcher (23) who was aiming for a major transfer in the posting system decided to contract with Angels. On August 8, agent Mr. Barzero announced. In addition, Angels also announced the following statement.',
'The U.S. major league, Angels announced that the shoulder number of Shohei Otani pitcher (23) whose entry was decided will be "17".', 
'The team, including the Carnegie Institute of America, announced that he found a super huge black hole that was born more than 13 billion years ago.According to the team, it is the oldest in observation history, the mass (weight) is about 800 million times of the sun. On the 7th, the paper was published in English science journal Nature, which is the result that leads to the elucidation of the evolution of the early universe.',
'In the universe about 13.1 billion light years away from the earth, the furthest and oldest black hole found so far was observed. It existed at the time when the universe was born, and existence at this time surprises scientists. The research was published in science journal Nature.'
] 

# 5-2-2.前処理
test_tfv = tfv_vector_fit.transform(test) # tf-idf計算
test_lsa = lsa.transform(test_tfv) # lsaで次元削減

# 5-2-3.予測
test_predict = clf.predict(test_lsa)
print(test_predict) # [0 0]


[0 0 1 1]


In [74]:
'''
yahooニュースから130億年以上前に誕生したブラックホールについての記事を２つチョイスして、google翻訳で英語にしたものを利用しています。
「1」というクラスに分類されています。先ほど確認した結果と比較すると問題なく分類されてそうです。
'''

# 5-3.発展(ブラックホール)
# 5-3-1.データの取り込み
blackhole = [
'The team, including the Carnegie Institute of America, announced that he found a super huge black hole that was born more than 13 billion years ago.According to the team, it is the oldest in observation history, the mass (weight) is about 800 million times of the sun. On the 7th, the paper was published in English science journal Nature, which is the result that leads to the elucidation of the evolution of the early universe.',
'In the universe about 13.1 billion light years away from the earth, the furthest and oldest black hole found so far was observed. It existed at the time when the universe was born, and existence at this time surprises scientists. The research was published in science journal Nature.'
]

# 5-3-2.前処理
blackhole_tfv = tfv_vector_fit.transform(blackhole) # tf-idf計算
blackhole_lsa = lsa.transform(blackhole_tfv) # lsaで次元削減

# 5-3-3.予測
blackhole_predict = clf.predict(blackhole_lsa)
print(blackhole_predict) # [1 1]

[1 1]


In [ ]:
'''
今回利用したデータセットが小さいこともあり、本モデルによる分類精度はあまりよくありません。

'''

